In [1]:
import cohere
import pandas as pd
from tqdm import tqdm

In [2]:
api_key = '0KeTU61A6Yci4y3En9gGOxBpI6aHzmx5d5LSQVrc'
co = cohere.Client(api_key)

In [3]:
examples = [
("Name Entity: John Smith", "John Smith is the CEO of XYZ Corporation."),
("Location Entity: Europe", "I would like to go on a Europe Trip"),
("Email Entity: support@example.com", "Please contact support@example.com for further assistance"),
("Phone No Entity: +1 (555) 123-4567", "You can reach our customer service at +1 (555) 123-4567"),
("Name Entity: James Cameron", "James Cameron congratulates Kevin Feige and Marvel!"),
("Date and Time Entity: January 15, 2024", "The meeting is scheduled for January 15, 2024."),
("Location Entity: London,New York" , "The flight from London to New York will depart at 11:00 PM on British Airways."),
("Name Entity: Elon Musk" , "The stock price of Tesla rose by 5% after Elon Musk announced a new Gigafactory in Berlin."),
("Phone No Entity: 555-1234" , "Call Dr. Smith at 555-1234 to schedule an appointment tomorrow at 10 AM."),
("Date and Time Entity: 1889" ,"The Eiffel Tower, completed in 1889, is a famous landmark located in Paris, France."),
]

In [4]:
#@title Create the prompt (Run this cell to execute required code) {display-mode: "form"}

class cohereExtractor():
    
    def __init__(self, examples, example_labels, labels, task_desciption, example_prompt):
        self.examples = examples
        self.example_labels = example_labels
        self.labels = labels
        self.task_desciption = task_desciption
        self.example_prompt = example_prompt

    def make_prompt(self, example):
        
        ## Define the prompt to extract entities from
        
        examples = self.examples + [example]
        labels = self.example_labels + [""]
        return (self.task_desciption +
                "\n---\n".join( [examples[i] + "\n" +
                                self.example_prompt + 
                                 labels[i] for i in range(len(examples))]))

    def extract(self, example):
        
        # Call the entity extraction endpoint
    
      extraction = co.generate(
          model='command',
          prompt=self.make_prompt(example),
          max_tokens=200,
          temperature=0.6,
          stop_sequences=["\n"])
      return(extraction.generations[0].text)


cohereMovieExtractor = cohereExtractor([e[1] for e in examples], [e[0] for e in examples], [],"", "extract the entity from text:")


In [13]:
#The entity that we are extracting from above text

set([e[0].split(':')[0] for e in examples])

{'Date and Time Entity',
 'Email Entity',
 'Location Entity',
 'Name Entity',
 'Phone No Entity'}

In [5]:
input_text = ["Welcome to the party! Sarah and Mark just arrived.","The conference will be held at the Grand Plaza Hotel, New York.","Introducing our new Marketing Manager, Alex Rodriguez.","Our company is hiring a Senior Software Engineer for the new project."]


In [6]:
results = []
for text in tqdm(input_text):
    try:
        extracted_text = cohereMovieExtractor.extract(text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)
results

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.70s/it]


[' Name Entity: Sarah, Mark',
 ' Location Entity: Grand Plaza Hotel, New York',
 ' Name Entity: Alex Rodriguez',
 ' Job Title Entity: Senior Software Engineer']